In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

# 데이타 전처리(Preprocessing)

- 데이타 클린징
- 결손값 처리 (Null, NaN 처리) 
- 데이터 인코딩 (레이블, 원핫인코딩)
- 데이터 스케일링
- 이상치 제거 
- 피처 선택, 추출 및 가공 

## 데이터 인코딩

- 머신러닝에서의 데이터는 숫자형으로 표현되어야한다. 
- 이를 위해 사이킷런에서 자동으로 숫자값으로 변환되는 API에는 레이블 인코딩과 원핫 인코딩이 있다 

# 레이블 인코딩(Label encoding)

: 사이킷런의 LabelEncoder 이용. 숫자형태로 변경 

- 모듈 임포트
from sklearn.preprocessing import LabelEncoder

- 인코딩 객체 생성 
encoder = LabelEncoder()

- 인코딩객체 fit => 인코딩.fit(타겟리스트) 
encoder.fit(items)

- 인코딩객체 변환및 저장 =>  인코딩.transform(타겟리스트)
labels = encoder.transform(items)

###  판다스의 apply() 활용

In [ ]:
items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

df = pd.DataFrame(np.array(items).reshape(-1,1), columns=['items'])
df.sort_values(by='items')

In [ ]:
result = df.sort_values(by='items')['items']
result_list = list(result.unique())
result_list

In [ ]:
def change(item) :
    result_list = ['TV', '냉장고', '믹서', '선풍기', '전자렌지', '컴퓨터']
    return result_list.index(item)

In [ ]:
change('믹서'), change('전자렌지')

In [ ]:
df['Label_encoder'] = df['items'].apply(change)

In [ ]:
df

### sklearn 의 LabelEncoder 활용

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
print(dir(LabelEncoder))

In [ ]:
items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

encoder = LabelEncoder()

encoder.fit(items)

labels = encoder.transform(items)
print('인코딩 변환값:',labels)

In [ ]:
items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

df = pd.DataFrame(np.array(items).reshape(-1,1), columns=['items'])

encoder = LabelEncoder()

encoder.fit(items)
 
labels = encoder.transform(items)

df['Label_encoder'] = labels

In [ ]:
df

### LabelEncoder() 의 메서드와 속성 활용 

- 인코딩객체.classes_ : 원본값
- 인코딩객체.inverse_transform() : 메소드 이용 원본값 보기 


In [ ]:
print(dir(encoder))

In [ ]:
print('인코딩 클래스:', encoder.classes_)

In [ ]:
print('디코딩 원본 값:', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

# 원-핫 인코딩(One-Hot encoding) 

- 원핫인코딩은 피처 값의 유형에 따라 새로운 피처를 추가해 고유값에 해당하는 컬럼에만 1을 표시하고 나머지 컬럼에는 0을 표시하는 방식이다. 하나만 1로 변환. 최종 2차원으로 변환



### 판다스의 get_dummies()를 이용한 원핫 인코딩

- pd.get_dummies(데이타프레임[컬럼명])
- pd.get_dummies(데이타프레임)

In [ ]:
items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

df = pd.DataFrame(np.array(items).reshape(-1,1), columns=['items'])
df

In [ ]:
pd.get_dummies(df)

In [ ]:
pd.get_dummies(df['items'])

# 타이타닉 생존자 예측 

###  데이타셋 
교재에서 제공하는 타이타닉 데이타 이용
data/titanic_book.csv

###  각 피처 설명 
* Passengerid: 탑승자 데이터 일련번호
* survived: 생존 여부, 0 = 사망, 1 = 생존
* Pclass: 티켓의 선실 등급, 1 = 일등석, 2 = 이등석, 3 = 삼등석
* sex: 탑승자 성별
* name: 탑승자 이름
* Age: 탑승자 나이
* sibsp: 같이 탑승한 형제자매 또는 배우자 인원수
* parch: 같이 탑승한 부모님 또는 어린이 인원수
* ticket: 티켓 번호
* fare: 요금
* cabin: 선실 번호
* embarked: 중간 정착 항구 C = Cherbourg, Q = Queenstown, S = Southampton

In [ ]:
titanic_df = pd.read_csv('data/titanic_book.csv')
titanic_df.head(3)

In [ ]:
print('\n ### train 데이터 정보 ###  \n')
print(titanic_df.info())

# null 결측치가 있는 피처 확인 - Age, Cabin, Embarked

##### NULL 컬럼들에 대한 처리 **

In [ ]:
# 평균값으로 대체 
titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)

titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)

print('데이터 세트 Null 값 갯수 ',titanic_df.isnull().sum().sum())

#### 각 피처의 값의 분포 value_counts()

In [ ]:
print(' Sex 값 분포 :\n',titanic_df['Sex'].value_counts())
print('\n Cabin 값 분포 :\n',titanic_df['Cabin'].value_counts())
print('\n Embarked 값 분포 :\n',titanic_df['Embarked'].value_counts())

In [ ]:
sns.countplot(x = 'Survived', data=titanic_df);

In [ ]:
sns.barplot(x='Sex', y='Survived', data=titanic_df)

In [ ]:
sns.barplot(x='Pclass', y='Survived', hue='Sex', ci=False, data=titanic_df)

### Sex, Embarked  전처리 
- LabelEncoder()로 변환

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
titanic_df['Sex'].value_counts()

In [ ]:
encoder = LabelEncoder()

encoder.fit(titanic_df['Sex'])

temp = encoder.transform(titanic_df['Sex'])

titanic_df['Sex'] = temp.reshape(-1, 1)

In [ ]:
titanic_df.head(2)

In [ ]:
titanic_df['Sex'].value_counts()

In [ ]:
# encoder = LabelEncoder()

encoder.fit(titanic_df['Embarked'])

temp = encoder.transform(titanic_df['Embarked'])

titanic_df['Embarked'] = temp.reshape(-1, 1)

In [ ]:
titanic_df['Embarked'].value_counts()

In [ ]:
titanic_df['Embarked'].value_counts()

In [ ]:
sns.countplot(x='Embarked', data=titanic_df);

### Cabin

In [ ]:
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df['Cabin'].value_counts()

In [ ]:
titanic_df.sample(3)

In [ ]:
encoder.fit(titanic_df['Cabin'])

temp = encoder.transform(titanic_df['Cabin'])

titanic_df['Cabin'] = temp.reshape(-1, 1)

In [ ]:
titanic_df['Cabin'].value_counts()

## 불필요한 컬럼 삭제 

In [ ]:
titanic_df.dtypes

In [ ]:
titanic_df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

In [ ]:
titanic_df.dtypes

### 학습데이타와 테스트데이타셋으로 분리

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
titanic_df.columns

In [ ]:
X_titanic_df = titanic_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked']]
y_titanic_df = titanic_df['Survived']

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=11, stratify=y_titanic_df)

### 모델에 적용

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [ ]:
model_kn = KNeighborsClassifier()
model_lr = LogisticRegression()
model_dt = DecisionTreeClassifier(random_state=11)
model_rf = RandomForestClassifier(random_state=11)


# KNeighborsClassifier 학습/예측/평가
model_kn.fit(X_train , y_train)
print('\n========= KNeighbors' )
print('Train score : ' ,model_kn.score(X_train, y_train))
print('Test score : ' , model_kn.score(X_test, y_test))
print(f'정확도: {accuracy_score(y_test, model_kn.predict(X_test)):.4f}')

# LogisticRegression 학습/예측/평가
model_lr.fit(X_train , y_train)
print('\n========= LogisticRegression' )
print('Train score : ' , model_lr.score(X_train, y_train))
print('Test score : ' , model_lr.score(X_test, y_test))
print(f'정확도: {accuracy_score(y_test, model_lr.predict(X_test)):.4f}')

# DecisionTreeClassifier 학습/예측/평가
model_dt.fit(X_train , y_train)
print('\n========= DecisionTree' )
print('Train score : ' , model_dt.score(X_train, y_train))
print('Test score : ' , model_dt.score(X_test, y_test))
print(f'정확도: {accuracy_score(y_test, model_dt.predict(X_test)):.4f}')

# RandomForest 학습/예측/평가
model_rf.fit(X_train , y_train)
print('\n========= RandomForest' )
print('Train score : ' , model_rf.score(X_train, y_train))
print('Test score : ' , model_rf.score(X_test, y_test))
print(f'정확도: {accuracy_score(y_test, model_rf.predict(X_test)):.4f}')


# Quiz

타이타닉 데이타셋에서 Age 컬럼을 아래의 함수를 이용하여 세분화하여 추가하고 원핫인코딩 형태 변경하여
<br> 모델의 데이타로 활용하여라. 

In [447]:
# 입력 age에 따라 구분값을 반환하는 함수 설정. DataFrame의 apply lambda식에 사용. 
def get_category(age):
    cat = ''
    if age <= -1: cat = 'Unknown'
    elif age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat

In [448]:
get_category(20)

'Student'

In [474]:
# 결과 예시 
titanic_df2.sample(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Adult,Baby,Child,Elderly,Student,Teenager,Young Adult
806,0,1,1,39.000000,0,0,0.0000,0,3,1,0,0,0,0,0,0
832,0,3,1,29.699118,0,0,7.2292,7,0,0,0,0,0,0,0,1
872,0,1,1,33.000000,0,0,5.0000,1,3,0,0,0,0,0,0,1
